## Определение стоимости автомобилей

### Задача

Цель проекта для нас подобрать наиболее подходящую по качеству и скорости модель для предсказания стоимости подержанного автомобиля по доступным характеристикам
Сервис по продаже автомобилей с пробегом разрабатывает приложение для привлечения новых клиентов. В нём можно быстро узнать рыночную стоимость своего автомобиля. 

В нашем распоряжении исторические данные: технические характеристики, комплектации и цены автомобилей. Нам нужно построить модель для определения стоимости. 

Заказчику важны:

- качество предсказания;
- скорость предсказания;
- время обучения.

### Подготовка данных

объекты имеют следующие параметры:
 - DateCrawled — дата скачивания анкеты из базы
 - VehicleType — тип автомобильного кузова
 - RegistrationYear — год регистрации автомобиля
 - Gearbox — тип коробки передач
 - Power — мощность (л. с.)
 - Model — модель автомобиля
 - Kilometer — пробег (км)
 - RegistrationMonth — месяц регистрации автомобиля
 - FuelType — тип топлива
 - Brand — марка автомобиля
 - NotRepaired — была машина в ремонте или нет
 - DateCreated — дата создания анкеты
 - NumberOfPictures — количество фотографий автомобиля
 - PostalCode — почтовый индекс владельца анкеты (пользователя)
 - LastSeen — дата последней активности пользователя <br>

Целевой признак
 - Price — цена (евро)

Импортируем необходимые библиотеки

In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error, make_scorer
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression

from catboost import CatBoostRegressor
import lightgbm as lgb

pd.options.mode.chained_assignment = None 

Загрузим таблицу

In [2]:
data = pd.read_csv('/datasets/autos.csv')

Приведем название колонок к нижнему регистру (так культурнее смотрится)

In [3]:
data.columns = data.columns.str.lower()
display(data.head(5))

,datecrawled,price,vehicletype,registrationyear,gearbox,power,model,kilometer,registrationmonth,fueltype,brand,notrepaired,datecreated,numberofpictures,postalcode,lastseen
0,2016-03-24 11:52:17,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,2016-03-24 00:00:00,0,70435,2016-04-07 03:16:57
1,2016-03-24 10:58:45,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes,2016-03-24 00:00:00,0,66954,2016-04-07 01:46:50
2,2016-03-14 12:52:21,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN,2016-03-14 00:00:00,0,90480,2016-04-05 12:47:46
3,2016-03-17 16:54:04,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,2016-03-17 00:00:00,0,91074,2016-03-17 17:40:17
4,2016-03-31 17:25:20,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,2016-03-31 00:00:00,0,60437,2016-04-06 10:17:21


Обозначим колонки с фичами и таргетом, также колонки с категоральными признаками

In [4]:
target = data['price']
#Для фич удалил колонки DateCrawled(дата скачивания анкеты из базы),RegistrationMonth(месяц регистрации автомобиля), 
#DateCreated(дата создания анкеты),PostalCode(почтовый индекс владельца анкеты),LastSeen(дата последней активности пользов.)
features_columns = (['vehicletype', 'registrationyear', 'gearbox','power', 'model', 'kilometer','fueltype', 'brand', 
                     'notrepaired', 'numberofpictures'])
cat_features = ['vehicletype', 'gearbox', 'model','fueltype', 'brand', 'notrepaired']

посмотрим полноту данных фичей

In [5]:
features = data[features_columns]
print(features.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 10 columns):
vehicletype         316879 non-null object
registrationyear    354369 non-null int64
gearbox             334536 non-null object
power               354369 non-null int64
model               334664 non-null object
kilometer           354369 non-null int64
fueltype            321474 non-null object
brand               354369 non-null object
notrepaired         283215 non-null object
numberofpictures    354369 non-null int64
dtypes: int64(4), object(6)
memory usage: 27.0+ MB
None


Заменим пропущенные значения

In [6]:
#Предположу, что в столбце "notrepaired" - пользователи не заполняли значения подразумевая, что машина была не бита
features['notrepaired'].fillna('no',inplace = True)
#Тип топлива угадать не получится, заменим значением unknown
features['fueltype'].fillna('unknown',inplace = True)
#Так как модель автомобиля имеет принципиально важное значение в стоимости автомобиля, дропнем строчки со значениями NaN
features.dropna(subset = ['model'],inplace = True)
#Тип коробки передач угадать не получится, заменим значением unknown
features['gearbox'].fillna('unknown',inplace = True)
#Тип кузова угадать не получится, заменим значением unknown
features['vehicletype'].fillna('unknown',inplace = True)

In [7]:
display(features.head())

,vehicletype,registrationyear,gearbox,power,model,kilometer,fueltype,brand,notrepaired,numberofpictures
0,unknown,1993,manual,0,golf,150000,petrol,volkswagen,no,0
2,suv,2004,auto,163,grand,125000,gasoline,jeep,no,0
3,small,2001,manual,75,golf,150000,petrol,volkswagen,no,0
4,small,2008,manual,69,fabia,90000,gasoline,skoda,no,0
5,sedan,1995,manual,102,3er,150000,petrol,bmw,yes,0


Зафиксируем данные (ЭТАП 1) после удаления nan, но до введения dummies (это нужно чтоб потом вытащить их для CatBoost)

In [8]:
features_smart = features

Подготовим строковые данные к обработке

In [9]:
features = pd.get_dummies(features, drop_first = True)

In [10]:
display(features.head())

,registrationyear,power,kilometer,numberofpictures,vehicletype_convertible,vehicletype_coupe,vehicletype_other,vehicletype_sedan,vehicletype_small,vehicletype_suv,...,brand_seat,brand_skoda,brand_smart,brand_subaru,brand_suzuki,brand_toyota,brand_trabant,brand_volkswagen,brand_volvo,notrepaired_yes
0,1993,0,150000,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,2004,163,125000,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,2001,75,150000,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0
4,2008,69,90000,0,0,0,0,0,1,0,...,0,1,0,0,0,0,0,0,0,0
5,1995,102,150000,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1


Разобъем таблицу с фичами на обучающую и тестувую части в соответстви 80 / 20

In [11]:
features_train, x = train_test_split(features, test_size = 0.4, random_state=12345)
features_valid, features_test = train_test_split(x, test_size = 0.5, random_state=12345)
target_train = target[features_train.index]
target_valid = target[features_valid.index]
target_test = target[features_test.index]

Зафиксируем данные (ЭТАП 2) это нужно чтоб потом вытащить их для CatBoost

In [12]:
features_train_smart = pd.DataFrame(features_smart, index = features_train.index)
features_valid_smart = pd.DataFrame(features_smart, index = features_valid.index)
features_test_smart = pd.DataFrame(features_smart, index = features_test.index)

In [13]:
display(features_train_smart.head())

,vehicletype,registrationyear,gearbox,power,model,kilometer,fueltype,brand,notrepaired,numberofpictures
68787,suv,2006,auto,232,q7,150000,gasoline,audi,no,0
318690,coupe,1989,manual,124,other,150000,petrol,honda,no,0
28935,small,2008,manual,75,colt,125000,petrol,mitsubishi,no,0
109327,wagon,1996,auto,197,e_klasse,150000,petrol,mercedes_benz,no,0
211417,wagon,1999,manual,82,astra,5000,gasoline,opel,no,0


Сделаем приближенные к реальным условия, обучим скалер только на трейновой ваборке, а применим ко всему датасету

In [14]:
scaler = StandardScaler()
scaler.fit(features_train)
features_train = scaler.transform(features_train)
features_valid = scaler.transform(features_valid)
features_test = scaler.transform(features_test)

### Обучение моделей

Модель линейной регрессии

In [15]:
%%time
model_regr = LinearRegression()
model_regr.fit(features_train,target_train)
answer_model_regr = model_regr.predict(features_valid)
rmse_model_regr = (mean_squared_error(answer_model_regr, target_valid))**0.5
print('RMSE простой модели линейной регрессии {:.0f}'.format(rmse_model_regr))

RMSE простой модели линейной регрессии 3185
CPU times: user 13.5 s, sys: 4.47 s, total: 18 s
Wall time: 18 s


Модель дерева решений

In [16]:
%%time
model_tree = DecisionTreeRegressor(random_state = 12345, max_depth = 19)
model_tree.fit(features_train,target_train)
answer_model_tree = model_tree.predict(features_valid)
rmse_model_tree = (mean_squared_error(answer_model_tree, target_valid))**0.5
print('RMSE простой модели дерева {:.0f}'.format(rmse_model_tree))

RMSE простой модели дерева 1966
CPU times: user 6.25 s, sys: 0 ns, total: 6.25 s
Wall time: 6.43 s


Модель случайного леса

In [17]:
%%time
model_forest = RandomForestRegressor(random_state = 12345, max_depth = 21,n_estimators = 27)
model_forest.fit(features_train,target_train)
answer_model_forest = model_forest.predict(features_valid)
rmse_model_forest = (mean_squared_error(answer_model_forest, target_valid))**0.5
print('RMSE простой модели леса деревьев {:.0f}'.format(rmse_model_forest))

RMSE простой модели леса деревьев 1735
CPU times: user 1min 51s, sys: 0 ns, total: 1min 51s
Wall time: 1min 52s


Средство библиотеки LightGBM

In [18]:
%%time
#Создадим датасет для lightgbm
lgb_train = lgb.Dataset(features_train, target_train)

params = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': {'l2', 'l1'},
    'num_leaves': 31,
    'learning_rate': 0.3
}
gbm = lgb.train(params, lgb_train, num_boost_round = 75)
answer_lgb = gbm.predict(features_valid, num_iteration=gbm.best_iteration)
rmse_lgb = (mean_squared_error(answer_lgb, target_valid))**0.5
print('RMSE библиотеки lightgbm {:.0f}'.format(rmse_lgb))

RMSE библиотеки lightgbm 1755
CPU times: user 31.4 s, sys: 311 ms, total: 31.7 s
Wall time: 31.9 s


Средство библиотеки CatBoost

In [19]:
%%time
model_catboost = CatBoostRegressor(iterations=200, learning_rate = 0.35, depth = 10)
model_catboost.fit(features_train_smart, target_train, cat_features=cat_features, verbose=10)
answer_model_catboost = model_catboost.predict(features_valid_smart)
rmse_model_catboost = (mean_squared_error(answer_model_catboost, target_valid))**0.5
answer_model_catboost_test = model_catboost.predict(features_test_smart)
print('RMSE библиотеки CatBoost {:.0f}'.format(rmse_model_catboost))

0:	learn: 3451.3129546	total: 824ms	remaining: 2m 43s
10:	learn: 1899.4112738	total: 7.82s	remaining: 2m 14s
20:	learn: 1786.7276577	total: 14.7s	remaining: 2m 5s
30:	learn: 1739.7502443	total: 22s	remaining: 1m 59s
40:	learn: 1703.6033450	total: 29.1s	remaining: 1m 52s
50:	learn: 1674.0806797	total: 36.6s	remaining: 1m 46s
60:	learn: 1649.7391910	total: 44.6s	remaining: 1m 41s
70:	learn: 1623.2694631	total: 52s	remaining: 1m 34s
80:	learn: 1604.1185489	total: 59.3s	remaining: 1m 27s
90:	learn: 1591.7761132	total: 1m 6s	remaining: 1m 20s
100:	learn: 1580.6616186	total: 1m 14s	remaining: 1m 13s
110:	learn: 1563.0923031	total: 1m 22s	remaining: 1m 5s
120:	learn: 1553.7441266	total: 1m 29s	remaining: 58.5s
130:	learn: 1546.3408747	total: 1m 37s	remaining: 51.4s
140:	learn: 1535.0731030	total: 1m 45s	remaining: 44.3s
150:	learn: 1526.5695359	total: 1m 53s	remaining: 36.7s
160:	learn: 1520.7204736	total: 2m	remaining: 29.3s
170:	learn: 1512.4108439	total: 2m 8s	remaining: 21.8s
180:	learn: 

### Таблица с параметрами и характеристиками всех моделей

In [20]:
result = pd.DataFrame(
 [['Модель линейной регрессии','не_менялись','RMSE',rmse_model_regr,'Wall time: 21.9 s'],
 ['Модель дерева решений','max_depth = 19','RMSE',rmse_model_tree,'Wall time: 6.92 s'],
 ['Модель случайного леса','max_depth = 21,n_estimators = 27','RMSE',rmse_model_forest,'Wall time: 2min 12s'],
 ['Средство библиотеки LightGBM',params,'RMSE', rmse_lgb,'Wall time: 51.2 s'],
 ['Средство библиотеки CatBoost','iterations=200, learning_rate = 0.35, depth = 10','RMSE',rmse_model_catboost,'Wall time: 4min 10s']], 
 columns = ['модель','гиперпараметры','метрика','показатель_метрики','время_обучения/ответа'])
display(result)

,модель,гиперпараметры,метрика,показатель_метрики,время_обучения/ответа
0,Модель линейной регрессии,не_менялись,RMSE,3184.715388,Wall time: 21.9 s
1,Модель дерева решений,max_depth = 19,RMSE,1966.230381,Wall time: 6.92 s
2,Модель случайного леса,"max_depth = 21,n_estimators = 27",RMSE,1735.389011,Wall time: 2min 12s
3,Средство библиотеки LightGBM,"{'boosting_type': 'gbdt', 'objective': 'regres...",RMSE,1755.489824,Wall time: 51.2 s
4,Средство библиотеки CatBoost,"iterations=200, learning_rate = 0.35, depth = 10",RMSE,1678.355787,Wall time: 4min 10s


### Анализ моделей

Поскольку заказчику важна скорость обучения/предсказаний, настройки всех моделей проводилась со следующей логикой: изменялись гиперпараметры и замерялась метрика RMSE, как только разница становилась незначительна (на 10 пунктов RMSE относительно предыдущего результата RMSE с предыдущим значением гиперпараметра) - изменения останавливались и гиперпараметр фиксировался<br> 
Наилучшим образом показала себя средство библиотеки CatBoost, сильно опережая остальных по качество (но правда и самое долгое)

> Средство библиотеки CatBoost, показавшее наилучшее качество, испытаем на тестовой выборке

In [21]:
%%time
answer_model_catboost_test = model_catboost.predict(features_test_smart)
rmse_model_catboost_test = (mean_squared_error(answer_model_catboost_test, target_test))**0.5
print('Финальный проверочный RMSE библиотеки CatBoost на тестовой выборке {:.0f}'.format(rmse_model_catboost_test))

Финальный проверочный RMSE библиотеки CatBoost на тестовой выборке 1726
CPU times: user 423 ms, sys: 10.3 ms, total: 433 ms
Wall time: 402 ms
